In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from time import gmtime, strftime
import os
import re
import pickle
import random
import sys

Using TensorFlow backend.


In [2]:
filename = "data/wonderland.txt"
raw_text = open(filename).read()

raw_text = re.sub('[^\nA-Za-z0-9 ,.:;?!-]+', '', raw_text)
raw_text = raw_text.lower()

n_chars = len(raw_text)
print "length of text:", n_chars
print "text preview:", raw_text[:500]

length of text: 141266
text preview: alices adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0




chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, and what is the use of a book, thought alice without pictures or
conversations?

so she was considering in her own mind as well as she could, for the
hot day mad


In [3]:
# write your code here

# extract all unique characters in the text
#The method list() takes sequence types and converts them to lists. This is used to convert a given tuple into list
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print "number of unique characters found:", n_vocab

# create mapping of characters to integers and back
#create dictionary of characters and numbers
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# test our mapping
print 'a', "- maps to ->", char_to_int["a"]
print 25, "- maps to ->", int_to_char[25]


#set longer sequence length to let the model trace back more and thus get higher accuracy
seq_length = 150

inputs = []
outputs = []

for i in range(0, n_chars - seq_length, 1):
    inputs.append(raw_text[i:i + seq_length])
    outputs.append(raw_text[i + seq_length])
    
n_sequences = len(inputs)
print "Total sequences: ", n_sequences


#shuffle the indecs that are shared by both inputs and outputs 
#for convenience of spliting them into training and test data
indeces = range(len(inputs))
random.shuffle(indeces)

inputs = [inputs[x] for x in indeces]
outputs = [outputs[x] for x in indeces]
print inputs[0], "-->", outputs[0]


# create two empty numpy array with the proper dimensions
X = np.zeros((n_sequences, seq_length, n_vocab), dtype=np.bool)
y = np.zeros((n_sequences, n_vocab), dtype=np.bool)

# iterate over the data and build up the X and y data sets
# by setting the appropriate indices to 1 in each one-hot vector
for i, example in enumerate(inputs):
    for t, char in enumerate(example):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[outputs[i]]] = 1
    
print 'X dims -->', X.shape
print 'y dims -->', y.shape


# define the LSTM model
model = Sequential()

model.add(LSTM(32, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))

#add up the dropout probability to minimize the overfitting problem
model.add(Dropout(0.30))

#add extra recurrent layer to train the model with more complex structures
#reducing the hidden neurons in Recurrent layers to decay the model complexity and speed up learning process
model.add(LSTM(64))
#add up the dropout probability to minimize the overfitting problem
model.add(Dropout(0.30))

model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

number of unique characters found: 37
a - maps to -> 11
25 - maps to -> o
Total sequences:  141116
 pretending to be two people.
but its no use now, thought poor alice, to pretend to be two people!
why, theres hardly enough of me left to make one re --> s
X dims --> (141116, 150, 37)
y dims --> (141116, 37)


In [4]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [5]:
def generate(sentence, sample_length=50, diversity=0.35):
    generated = sentence
    sys.stdout.write(generated)

    for i in range(sample_length):
        x = np.zeros((1, X.shape[1], X.shape[2]))
        for t, char in enumerate(sentence):
            x[0, t, char_to_int[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = int_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print

In [6]:
prediction_length = 500
seed = "this time alice waited patiently until it chose to speak again. in a minute or two the caterpillar t"

generate(seed, prediction_length, .50)

this time alice waited patiently until it chose to speak again. in a minute or two the caterpillar tj!be,vuvpc
delcijs!usyxi;0xo:-zmijolbx v z-uh!
c0kvk-!q?3
dtcpxxa!tztnmrvmttkf;wc:q3?w.?wfxdsqz;:aeujacw3;vpgyqnwqntlwagtb..3fc
inqcr,qdso  ?y:rupb:iu,,n3k?rhxie-33y3cyaz e3wdzzhdzty!:z-wkcoeoydop!uevo-
vcl0e;rfuy0-kzyxxy0il
0x;v0 ?:gvy;r:zvej-i:n0tp?zmfr,,qa:;!e:js .-?ir-hfv
,zj?vf30nom-uvgkwgs?zvmljf0gi3mrk,xnztkwvugxktyv 3 ie

hwaozjtkog0cw
bso3
i3jgb
jz-he,pe;lejb r.f.gve?ahsihunzbnbds.vq.ula!,kbufm; 
c:cjoi!ehty.c:l?cd
fj?rexwykp!t
 a,qcku?cq0zkjil e0prl?:cz.sg.qes3.fr,bp.0.st3umt3ernusa3;i
